In [1]:
!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html
!pip install -q dive-into-graphs

In [2]:
!pip install -q toolz
!pip install -q wandb

In [3]:
%load_ext autoreload
%autoreload 2
import os
import json
import argparse
import pickle 

import numpy as np
import pandas as pd
import torch

from molecule_optimizer.externals.fast_jtnn.datautils import SemiMolTreeFolder, SemiMolTreeFolderTest
from molecule_optimizer.runner.semi_jtvae import SemiJTVAEGeneratorPredictor
from torch_geometric.data import DenseDataLoader

import rdkit

lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

import warnings
warnings.filterwarnings("ignore")

import math

In [4]:
conf = json.load(open("training/configs/rand_gen_zinc250k_config_dict.json"))

In [5]:
csv = pd.read_csv("ZINC_310k.csv")

In [6]:
smiles = csv['SMILES']

In [7]:
# smiles = smiles[:60000]

In [8]:
# labels = torch.tensor(csv['LogP'][:60000]).float()

In [7]:
labels = torch.tensor(csv['LogP']).float()

In [ ]:
# N_TEST = 10000
N_TEST = 200
VAL_FRAC = 0.05
chem_prop = "LogP"
load_epoch = 5000

In [8]:
# if 'runner.xml' not in os.listdir("."):
#     runner = SemiJTVAEGeneratorPredictor(smiles)
#     processed_smiles, processed_idxs = SemiJTVAEGeneratorPredictor.preprocess(smiles) 
#     with open('runner.xml', 'wb') as f:
#         pickle.dump(runner, f)

100%|██████████| 62/62 [1:06:51<00:00, 64.70s/it] 


In [11]:
# if 'runner_20.xml' not in os.listdir("."):
#     runner = SemiJTVAEGeneratorPredictor(smiles)
#     processed_smiles, processed_idxs = SemiJTVAEGeneratorPredictor.preprocess(smiles) 
#     with open('runner_20.xml', 'wb') as f:
#         pickle.dump(runner, f)

100%|██████████| 12/12 [19:09<00:00, 95.82s/it]


In [10]:
with open('saved/runner_LogP_50_1_iter_5000.xml', 'rb') as f:
    runner = pickle.load(f)

In [11]:
runner.get_model(
    "rand_gen",
    {
        "hidden_size": conf["model"]["hidden_size"],
        "latent_size": conf["model"]["latent_size"],
        "depthT": conf["model"]["depthT"],
        "depthG": conf["model"]["depthG"],
        "label_size": 1,
        "label_mean": float(torch.mean(labels)),
        "label_var": float(torch.var(labels)),
    },
)

In [11]:
labels = runner.get_processed_labels(labels, processed_idxs)
preprocessed = processed_smiles

In [12]:

perm_id=np.random.permutation(len(labels))

X_train = preprocessed[perm_id[N_TEST:]]
X_train_smiles = smiles[perm_id[N_TEST:]]
L_train = torch.tensor(labels.numpy()[perm_id[N_TEST:]])


X_test = preprocessed[perm_id[:N_TEST]]
X_test_smiles = smiles[perm_id[:N_TEST]]
L_test = torch.tensor(labels.numpy()[perm_id[:N_TEST]])

val_cut = math.floor(len(X_train) * VAL_FRAC)

X_Val = X_train[:val_cut]
X_Val_smiles = X_train_smiles[:val_cut]
L_Val = L_train[:val_cut]

X_train = X_train[val_cut :]
X_train_smiles = X_train_smiles[val_cut :]
L_train = L_train[val_cut :]

with open("train_smiles_" + chem_prop + "_50_1.npy", 'wb') as f:
    np.save(f, X_train_smiles)

with open("test_smiles_" + chem_prop + "_50_1.npy", 'wb') as f:
    np.save(f, X_test_smiles)

with open("validation_smiles_" + chem_prop + "_50_1.npy", 'wb') as f:
    np.save(f, X_Val_smiles)

#save preproccessed

with open("train_" + chem_prop + "_50_1.npy", 'wb') as f:
    np.save(f, X_train)

with open("test_" + chem_prop + "_50_1.npy", 'wb') as f:
    np.save(f, X_test)

with open("validation_" + chem_prop + "_50_1.npy", 'wb') as f:
    np.save(f, X_Val)

#Save labels

torch.save(L_train, "L_train_" + chem_prop + "_50_1.pt")

torch.save(L_test, "L_test_" + chem_prop + "_50_1.pt")

torch.save(L_Val, "L_Val_" + chem_prop + "_50_1.pt")

In [ ]:
L_train = torch.load("L_train_" + chem_prop + "_50_1.pt")
L_test = torch.load("L_test_" + chem_prop + "_50_1.pt")
L_Val = torch.load("L_Val_" + chem_prop + "_50_1.pt")

with open("train_" + chem_prop + "_50_1.npy", 'rb') as f:
    X_train = np.load(f, allow_pickle=True)

with open("test_" + chem_prop + "_50_1.npy", 'rb') as f:
    X_test = np.load(f, allow_pickle=True)

with open("validation_" + chem_prop + "_50_1.npy", 'rb') as f:
    X_Val = np.load(f, allow_pickle=True)

In [ ]:
# print("Training model...")
# runner.train_gen_pred(
#     X_train,
#     L_train,
#     X_test,
#     L_test,
#     X_Val,
#     L_Val,
#     load_epoch= 1000,
#     lr=conf["lr"],
#     anneal_rate=conf["anneal_rate"],
#     clip_norm=conf["clip_norm"],
#     num_epochs=conf["num_epochs"],
#     alpha=conf["alpha"],
#     max_alpha=conf["max_alpha"],
#     step_alpha=conf["step_alpha"],
#     beta=conf["beta"],
#     max_beta=conf["max_beta"],
#     step_beta=conf["step_beta"],
#     anneal_iter=conf["anneal_iter"],
#     alpha_anneal_iter=conf["alpha_anneal_iter"],
#     kl_anneal_iter=conf["kl_anneal_iter"],
#     print_iter=100,
#     save_iter= 1000,
#     batch_size=conf["batch_size"],
#     num_workers=conf["num_workers"],
#     label_pct=0.5,
#     chem_prop = "LogP"
# )

Training model...
Model #Params: 4732K
[Train][1100] Alpha: 0.000, Beta: 0.000, Loss: 40.79, KL: 679.81, MAE: 0.85188, Word Loss: 28.13, Topo Loss: 7.37, Assm Loss: 5.29, Pred Loss: 0.84, Word: 75.06, Topo: 94.72, Assm: 75.16, PNorm: 153.08, GNorm: 47.52
[Train][1200] Alpha: 0.000, Beta: 0.000, Loss: 39.47, KL: 717.46, MAE: 0.84108, Word Loss: 27.26, Topo Loss: 6.95, Assm Loss: 5.26, Pred Loss: 0.81, Word: 76.19, Topo: 95.11, Assm: 75.32, PNorm: 157.58, GNorm: 35.72
[Train][1300] Alpha: 0.000, Beta: 0.000, Loss: 37.01, KL: 750.64, MAE: 0.85575, Word Loss: 25.91, Topo Loss: 6.35, Assm Loss: 4.75, Pred Loss: 0.83, Word: 76.89, Topo: 95.61, Assm: 77.72, PNorm: 161.90, GNorm: 50.00
[Train][1400] Alpha: 0.000, Beta: 0.000, Loss: 36.21, KL: 796.35, MAE: 0.81510, Word Loss: 25.03, Topo Loss: 6.43, Assm Loss: 4.75, Pred Loss: 0.78, Word: 77.64, Topo: 95.35, Assm: 77.52, PNorm: 164.86, GNorm: 42.02
[Train][1500] Alpha: 0.000, Beta: 0.000, Loss: 34.63, KL: 834.32, MAE: 0.76559, Word Loss: 23.85,

In [ ]:
print("Training model...")
runner.train_gen_pred_supervised(
    X_train,
    L_train,
    X_test,
    L_test,
    X_Val,
    L_Val,
    load_epoch= 0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    max_alpha=conf["max_alpha"],
    step_alpha=conf["step_alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    alpha_anneal_iter=conf["alpha_anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter= conf["save_iter"],
    batch_size=conf["batch_size"],
    num_workers=conf["num_workers"],
    label_pct=0.5,
    chem_prop = "LogP"
)

Training model...
Initializing...
Model #Params: 5207K
[Train][100] Alpha: 0.000, Beta: 0.000, Loss: 61.27, KL: 185.61, MAE: 1.53639, Word Loss: 45.94, Topo Loss: 10.87, Assm Loss: 4.46, Pred Loss: 2.62, Word: 28.95, Topo: 83.15, Assm: 55.49, PNorm: 103.46, GNorm: 44.32
[Train][200] Alpha: 0.000, Beta: 0.000, Loss: 38.47, KL: 169.67, MAE: 1.59289, Word Loss: 28.09, Topo Loss: 6.18, Assm Loss: 4.20, Pred Loss: 2.89, Word: 51.02, Topo: 91.55, Assm: 58.88, PNorm: 107.94, GNorm: 40.76
[Train][300] Alpha: 0.000, Beta: 0.000, Loss: 33.83, KL: 172.82, MAE: 1.63952, Word Loss: 24.30, Topo Loss: 5.48, Assm Loss: 4.05, Pred Loss: 2.99, Word: 58.74, Topo: 92.44, Assm: 61.21, PNorm: 111.25, GNorm: 23.04
[Train][400] Alpha: 0.000, Beta: 0.000, Loss: 30.26, KL: 199.53, MAE: 1.65205, Word Loss: 21.46, Topo Loss: 4.89, Assm Loss: 3.90, Pred Loss: 3.00, Word: 63.16, Topo: 93.10, Assm: 61.72, PNorm: 114.16, GNorm: 28.80
[Train][500] Alpha: 0.000, Beta: 0.000, Loss: 28.41, KL: 214.00, MAE: 1.74623, Word 